# GAN Loss

## How to Use Different GAN Loss Functions

In [1]:
# example of lsgan for mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Activation
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.initializers import RandomNormal
from matplotlib import pyplot

### Discriminator Model

In [ ]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # define model
  model = Sequential()
  # downsample to 14x14
  model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  # downsample to 7x7
  model.add(Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)) 
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  # classifier
  model.add(Flatten())
  model.add(Dense(1, activation='linear', kernel_initializer=init)) # compile model with L2 loss
  model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5)) 
  return model

### Generator Model

In [ ]:
# define the standalone generator model
def define_generator(latent_dim):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # define model
  model = Sequential()
  # foundation for 7x7 image
  n_nodes = 256 * 7 * 7
  model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim)) 
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Reshape((7, 7, 256)))
  # upsample to 14x14
  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  # upsample to 28x28
  model.add(Conv2DTranspose(64, (4,4), strides=(2,2), padding='same',
  kernel_initializer=init))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  # output 28x28x1
  model.add(Conv2D(1, (7,7), padding='same', kernel_initializer=init)) 
  model.add(Activation('tanh'))
  return model

### Define the GAN

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
  # make weights in the discriminator not trainable discriminator.trainable = False
  # connect them
  model = Sequential()
  # add generator
  model.add(generator)
  # add the discriminator
  model.add(discriminator)
  # compile model with L2 loss
  model.compile(loss='mse', optimizer=Adam(lr=0.0002, beta_1=0.5)) 
  return model